In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from tqdm import tqdm
import cupy as cp
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Ridge, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error
from catboost import CatBoostRegressor


calendar_df = pd.read_csv('/kaggle/input/rohlik-sales-forecasting-challenge-v2/calendar.csv')
inventory_df = pd.read_csv('/kaggle/input/rohlik-sales-forecasting-challenge-v2/inventory.csv')
train_df = pd.read_csv('/kaggle/input/rohlik-sales-forecasting-challenge-v2/sales_train.csv')
test_df = pd.read_csv('/kaggle/input/rohlik-sales-forecasting-challenge-v2/sales_test.csv')
df5 = pd.read_csv('/kaggle/input/rohlik-sales-forecasting-challenge-v2/solution.csv')
weights_df = pd.read_csv('/kaggle/input/rohlik-sales-forecasting-challenge-v2/test_weights.csv')


In [ ]:
train_df.info()

In [ ]:
test_columns = list(test_df.columns)
keep_columns =  list(train_df.columns)
print(test_columns)
keep_columns 

# Data preprocessing

In [ ]:
 #Định dạng lại format date
calendar_df['date'] = pd.to_datetime(calendar_df['date'])
train_df['date'] = pd.to_datetime(train_df['date'])
test_df['date'] = pd.to_datetime(test_df['date'])


In [ ]:
Frankfurt_1 = calendar_df.query('date >= "2020-08-01 00:00:00" and warehouse =="Frankfurt_1"')
Prague_2 = calendar_df.query('date >= "2020-08-01 00:00:00" and warehouse =="Prague_2"')
Brno_1 = calendar_df.query('date >= "2020-08-01 00:00:00" and warehouse =="Brno_1"')
Munich_1 = calendar_df.query('date >= "2020-08-01 00:00:00" and warehouse =="Munich_1"')
Prague_3 = calendar_df.query('date >= "2020-08-01 00:00:00" and warehouse =="Prague_3"')
Prague_1 = calendar_df.query('date >= "2020-08-01 00:00:00" and warehouse =="Prague_1"')
Budapest_1 = calendar_df.query('date >= "2020-08-01 00:00:00" and warehouse =="Budapest_1"')
def process_calendar(df):
    """
    - days_to_holiday
    - days_to_shops_closed
    - day_after_closing
    - long_weekend
    - weekday
    - ...
    """
    df = df.sort_values('date').reset_index(drop=True)


    # 4. long_weekend
    df['long_weekend'] = (
        (df['shops_closed'] == 1) & (df['shops_closed'].shift(1) == 1)
    ).astype(int)

    # 5. weekday
    df['weekday'] = df['date'].dt.weekday  # 0 (понедельник) - 6 (воскресенье)


    # 7. holiday season
    df['holiday_season'] = df['date'].apply(lambda x: 1 if (x.month == 12 and x.day >= 20) or (x.month == 1 and x.day <= 5) else 0)

    # 8. week of month
    df['week_of_month'] = df['date'].apply(lambda x: (x.day - 1) // 7 + 1)

    # 9. quarter
    df['quarter'] = df['date'].dt.quarter

    # 10. is weekend
    df['is_weekend'] = df['date'].dt.weekday.isin([5, 6]).astype(int)

    return df


dfs = ['Frankfurt_1', 'Prague_2', 'Brno_1', 'Munich_1', 'Prague_3', 'Prague_1', 'Budapest_1']
processed_dfs = [process_calendar(globals()[df]) for df in dfs]

calendar_extended = pd.concat(processed_dfs).sort_values('date').reset_index(drop=True)
print(calendar_extended.isna().sum())

In [ ]:
train_calendar = train_df.merge(calendar_extended, on=['date', 'warehouse'], how='left')
train_inventory = train_calendar.merge(inventory_df, on=['unique_id', 'warehouse'], how='left')
train_df = train_inventory.merge(weights_df, on=['unique_id'], how='left')

test_calendar = test_df.merge(calendar_extended, on=['date', 'warehouse'], how='left')
test_df = test_calendar.merge(inventory_df, on=['unique_id', 'warehouse'], how='left')

In [ ]:
print("Number of unique day:",train_df['date'].nunique())
train_df.head()

In [ ]:
print(f"Shape")
print(f"train: {train_df.shape}")
print(f"test: {test_df.shape}")

In [ ]:
train_df.isnull().sum()

In [ ]:
train_df.loc[train_df.sales.isnull(),:].reset_index().groupby(['warehouse'],observed=False). \
agg(size=('warehouse','size'),
    min_date=('date','min'),
    max_date=('date','max'),
    days = ('date', lambda x: x.max() - x.min()),
    split_date=('date', lambda x: list(np.unique(np.unique(x.dt.strftime('%Y-%m-%d'))))) 
   ).dropna()

In [ ]:
train_df['sales'] = train_df['sales'].fillna(0)
train_df['total_orders'] = train_df['total_orders'].fillna(0)
train_df['sell_price_main'] = train_df['sell_price_main'].interpolate()

In [ ]:
train_df.isnull().sum()

In [ ]:
print("Số lượng NaN trong holiday_name theo holiday:")
print(train_df.groupby('holiday')['holiday_name'].apply(lambda x: x.isna().sum()))

In [ ]:
# Kiểm tra tổng số hàng và số NaN trong holiday_name
print("Tổng số hàng trong train_df:", len(train_df))
print("Số lượng NaN trong holiday_name:", train_df['holiday_name'].isna().sum())

# Kiểm tra số lượng NaN trong holiday_name theo holiday
print("\nSố lượng NaN trong holiday_name theo holiday:")
print(train_df.groupby('holiday')['holiday_name'].apply(lambda x: x.isna().sum()))

# Lọc các ngày lễ (holiday == 1) có holiday_name là NaN
missing_holidays = train_df[(train_df['holiday'] == 1) & (train_df['holiday_name'].isna())][['date', 'warehouse', 'holiday', 'holiday_name']]

# Hiển thị kết quả
print("\nCác ngày lễ thiếu tên (holiday == 1, holiday_name là NaN):")
print(missing_holidays)

# Thống kê số lượng ngày lễ thiếu tên theo warehouse
print("\nSố lượng ngày lễ thiếu tên theo warehouse:")
print(missing_holidays.groupby('warehouse').size())

# Thống kê các ngày duy nhất bị thiếu tên
print("\nCác ngày duy nhất bị thiếu tên lễ:")
print(missing_holidays['date'].dt.strftime('%Y-%m-%d').unique())

# Fill những ngày bị miss


In [ ]:
# Danh sách ngày lễ (giữ nguyên từ code của bạn)
czech_holiday = [
    (['03/31/2024', '04/09/2023', '04/17/2022', '04/04/2021', '04/12/2020'], 'Easter Day'),
    (['05/12/2024', '05/10/2020', '05/09/2021', '05/08/2022', '05/14/2023'], "Mother's Day"),
]
brno_holiday = [
    (['03/31/2024', '04/09/2023', '04/17/2022', '04/04/2021', '04/12/2020'], 'Easter Day'),
    (['05/12/2024', '05/10/2020', '05/09/2021', '05/08/2022', '05/14/2023'], "Mother's Day"),
]
budapest_holidays = []
munich_holidays = [
    (['03/30/2024', '04/08/2023', '04/16/2022', '04/03/2021'], 'Holy Saturday'),
]
frank_holidays = [
    (['03/30/2024', '04/08/2023', '04/16/2022', '04/03/2021'], 'Holy Saturday'),
    (['05/12/2024', '05/14/2023', '05/08/2022', '05/09/2021'], "Mother's Day"),
]

# Hàm fill_loss_holidays (giữ nguyên)
def fill_loss_holidays(df_fill, warehouses, holidays):
    df = df_fill.copy()
    for item in holidays:
        dates, holiday_name = item
        generated_dates = [datetime.strptime(date, '%m/%d/%Y').strftime('%Y-%m-%d') for date in dates]
        for generated_date in generated_dates:
            df.loc[(df['warehouse'].isin(warehouses)) & (df['date'] == generated_date), 'holiday'] = 1
            df.loc[(df['warehouse'].isin(warehouses)) & (df['date'] == generated_date), 'holiday_name'] = holiday_name
    return df

# Kiểm tra NaN trong holiday_name trước khi xử lý
print("Số lượng NaN trong holiday_name (train_df) trước khi xử lý:", train_df['holiday_name'].isna().sum())
print("\nSố lượng NaN trong holiday_name theo holiday:")
print(train_df.groupby('holiday')['holiday_name'].apply(lambda x: x.isna().sum()))

# Điền ngày lễ vào train_df
train_df = fill_loss_holidays(df_fill=train_df, warehouses=['Prague_1', 'Prague_2', 'Prague_3'], holidays=czech_holiday)
train_df = fill_loss_holidays(df_fill=train_df, warehouses=['Brno_1'], holidays=brno_holiday)
train_df = fill_loss_holidays(df_fill=train_df, warehouses=['Munich_1'], holidays=munich_holidays)
train_df = fill_loss_holidays(df_fill=train_df, warehouses=['Frankfurt_1'], holidays=frank_holidays)
train_df = fill_loss_holidays(df_fill=train_df, warehouses=['Budapest_1'], holidays=budapest_holidays)

# Điền các NaN còn lại trong holiday_name bằng "No Holiday"
train_df['holiday_name'] = train_df['holiday_name'].fillna("No Holiday")

# Kiểm tra NaN sau khi xử lý
print("\nSố lượng NaN trong holiday_name (train_df) sau khi xử lý:", train_df['holiday_name'].isna().sum())
print("\nPhân bố giá trị trong holiday_name:")
print(train_df['holiday_name'].value_counts())
print("\nGiá trị holiday_name khi holiday == 1:")
print(train_df[train_df['holiday'] == 1]['holiday_name'].value_counts())

# Xử lý test_df (tương tự)
print("\nSố lượng NaN trong holiday_name (test_df) trước khi xử lý:", test_df['holiday_name'].isna().sum())
test_df = fill_loss_holidays(df_fill=test_df, warehouses=['Prague_1', 'Prague_2', 'Prague_3'], holidays=czech_holiday)
test_df = fill_loss_holidays(df_fill=test_df, warehouses=['Brno_1'], holidays=brno_holiday)
test_df = fill_loss_holidays(df_fill=test_df, warehouses=['Munich_1'], holidays=munich_holidays)
test_df = fill_loss_holidays(df_fill=test_df, warehouses=['Frankfurt_1'], holidays=frank_holidays)
test_df = fill_loss_holidays(df_fill=test_df, warehouses=['Budapest_1'], holidays=budapest_holidays)

# Điền NaN còn lại trong test_df
test_df['holiday_name'] = test_df['holiday_name'].fillna("No Holiday")

# Kiểm tra NaN sau khi xử lý
print("\nSố lượng NaN trong holiday_name (test_df) sau khi xử lý:", test_df['holiday_name'].isna().sum())
print("\nPhân bố giá trị trong holiday_name (test_df):")
print(test_df['holiday_name'].value_counts())

# # Lưu dữ liệu
# train_df.to_csv('train_df_cleaned_holiday_name.csv', index=False)
# test_df.to_csv('test_df_cleaned_holiday_name.csv', index=False)

# xử lý outlier trong sales

In [ ]:
Q1 = np.log1p(train_df["sales"]).quantile(0.25)
Q3 = np.log1p(train_df["sales"]).quantile(0.75)
IQR = Q3 - Q1
lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

train_df = train_df[(np.log1p(train_df["sales"]) >= lower) & (np.log1p(train_df["sales"]) <= upper)]

In [ ]:
plt.figure(figsize=(6, 5))

sns.histplot(x=np.log1p(train_df['sales']), bins=100, kde=True)

plt.title(f'Log histplot of sales')
plt.xlabel('sales')
plt.ylabel('Frequency')

plt.show()

# xử lý discount <0 

In [ ]:
train_df.loc[train_df['type_0_discount'] < 0, 'type_0_discount'] = 0
train_df.loc[train_df['type_4_discount'] < 0, 'type_4_discount'] = 0
train_df.loc[train_df['type_6_discount'] < 0, 'type_6_discount'] = 0

In [ ]:
from sklearn.preprocessing import MinMaxScaler
cols_to_scale = ['total_orders', 'sell_price_main']

scaler = MinMaxScaler()
train_df[cols_to_scale] = scaler.fit_transform(train_df[cols_to_scale])

In [ ]:


# Kiểm tra các cột còn lại trong train_df
print("Các cột còn lại trong train_df:", train_df.columns.tolist())
print("Danh sách cột giữ lại (keep_columns):", keep_columns)

In [ ]:
train_df.head()

In [ ]:
train_df.info()

# Feature engineering

In [ ]:
# Đảm bảo tqdm hoạt động với pandas
tqdm.pandas()

# Kiểm tra dữ liệu train_df trước khi xử lý
print("Kiểm tra NaN trong train_df trước khi xử lý:")
print(train_df[['date', 'warehouse', 'total_orders']].isna().sum())
print("\nĐịnh dạng cột date:", train_df['date'].dtype)

# Đảm bảo cột date là datetime
train_df['date'] = pd.to_datetime(train_df['date'])
test_df['date'] = pd.to_datetime(test_df['date'])

# Tính total_orders_median
train_df["total_orders_median"] = train_df.groupby(["date", "warehouse"])["total_orders"].transform("median")

# Tính total_orders_med_diff
train_df["total_orders_med_diff"] = train_df["total_orders"] - train_df["total_orders_median"]

# Tính max và min demand cho mỗi warehouse
df_warehouse_limits = train_df.groupby("warehouse")["total_orders"].agg(["max", "min"])

# Kiểm tra trường hợp max = min để tránh chia cho 0
print("\nKiểm tra max và min của total_orders theo warehouse:")
print(df_warehouse_limits)
if (df_warehouse_limits["max"] == df_warehouse_limits["min"]).any():
    print("Cảnh báo: Một số warehouse có max = min, có thể gây lỗi khi tính warehouse_demand.")

# Tính warehouse_demand (sử dụng vector hóa thay vì progress_apply để tối ưu hiệu suất)
train_df["warehouse_demand"] = train_df.apply(
    lambda x: (
        (x["total_orders_median"] - df_warehouse_limits.loc[x["warehouse"], "min"]) /
        (df_warehouse_limits.loc[x["warehouse"], "max"] - df_warehouse_limits.loc[x["warehouse"], "min"])
    ) if (df_warehouse_limits.loc[x["warehouse"], "max"] != df_warehouse_limits.loc[x["warehouse"], "min"]) else 0,
    axis=1
)

# Kiểm tra NaN trong các cột mới
print("\nKiểm tra NaN trong train_df sau khi xử lý:")
print(train_df[["total_orders_median", "total_orders_med_diff", "warehouse_demand"]].isna().sum())

# Kiểm tra phân bố của các cột mới
print("\nThống kê mô tả cho các cột mới:")
print(train_df[["total_orders_median", "total_orders_med_diff", "warehouse_demand"]].describe())

# Áp dụng tương tự cho test_df (nếu cần)
print("\nKiểm tra NaN trong test_df trước khi xử lý:")
print(test_df[['date', 'warehouse', 'total_orders']].isna().sum())

# Tính total_orders_median cho test_df
test_df["total_orders_median"] = test_df.groupby(["date", "warehouse"])["total_orders"].transform("median")

# Tính total_orders_med_diff
test_df["total_orders_med_diff"] = test_df["total_orders"] - test_df["total_orders_median"]

# Tính warehouse_demand (sử dụng df_warehouse_limits từ train_df để đảm bảo chuẩn hóa nhất quán)
test_df["warehouse_demand"] = test_df.apply(
    lambda x: (
        (x["total_orders_median"] - df_warehouse_limits.loc[x["warehouse"], "min"]) /
        (df_warehouse_limits.loc[x["warehouse"], "max"] - df_warehouse_limits.loc[x["warehouse"], "min"])
    ) if (df_warehouse_limits.loc[x["warehouse"], "max"] != df_warehouse_limits.loc[x["warehouse"], "min"]) else 0,
    axis=1
)

# Kiểm tra NaN trong test_df
print("\nKiểm tra NaN trong test_df sau khi xử lý:")
print(test_df[["total_orders_median", "total_orders_med_diff", "warehouse_demand"]].isna().sum())



In [ ]:
# Total and average sales per category
category_sales = train_df.groupby('L1_category_name_en')['sales'].agg(['sum', 'mean']).reset_index()
category_sales.rename(columns={'sum': 'category_sales_sum', 'mean': 'category_sales_avg'}, inplace=True)
# Merge into training and test datasets
train_df = train_df.merge(category_sales, on='L1_category_name_en', how='left')
test_df = test_df.merge(category_sales, on='L1_category_name_en', how='left')

# Convert date to datetime and extract components
for df in [train_df, test_df]:
    df['date'] = pd.to_datetime(df['date'])
    df['month'] = df['date'].dt.month
    df['day_of_week'] = df['date'].dt.dayofweek
    df['week_of_year'] = df['date'].dt.isocalendar().week

# 1.Total and average orders per category
cat_order_stats = train_df.groupby('L1_category_name_en')['total_orders'].agg(['sum','mean']).reset_index()
cat_order_stats.rename(columns={'sum': 'category_orders_sum', 'mean': 'category_orders_avg'}, inplace=True)

# 2. Merge các chỉ số này vào train_df
train_df = train_df.merge(cat_order_stats,
                          on='L1_category_name_en',
                          how='left')

# 3. Merge các chỉ số này vào test_df
test_df = test_df.merge(cat_order_stats,
                        on='L1_category_name_en',
                        how='left')

In [ ]:
# Calculate the maximum discount applied in Sales_Train
discount_columns = [f'type_{i}_discount' for i in range(7)]
train_df['max_discount'] = train_df[discount_columns].max(axis=1)

# Calculate the maximum discount applied in Sales_Test
discount_columns_test = [f'type_{i}_discount' for i in range(7)]
test_df['max_discount'] = test_df[discount_columns_test].max(axis=1)

In [ ]:
train_df.info()

In [ ]:
from sklearn.preprocessing import LabelEncoder
# Kiểm tra và chuyển cột date thành datetime
if 'date' not in train_df.columns:
    raise ValueError("Cột 'date' không tồn tại trong train_df")
if 'date' not in test_df.columns:
    raise ValueError("Cột 'date' không tồn tại trong test_df")

train_df['date'] = pd.to_datetime(train_df['date'], errors='coerce')
test_df['date'] = pd.to_datetime(test_df['date'], errors='coerce')

# Kiểm tra giá trị datetime hợp lệ
if train_df['date'].isna().all():
    raise ValueError("Cột 'date' trong train_df không chứa giá trị datetime hợp lệ")
if test_df['date'].isna().all():
    raise ValueError("Cột 'date' trong test_df không chứa giá trị datetime hợp lệ")

# Thêm các đặc trưng sin và cos cho train_df
train_df["day"] = train_df["date"].dt.day
train_df["month"] = train_df["date"].dt.month
train_df["year"] = train_df["date"].dt.year
train_df['year_sin'] = np.sin(2 * np.pi * train_df['year'] / train_df['year'].max())
train_df['year_cos'] = np.cos(2 * np.pi * train_df['year'] / train_df['year'].max())
train_df['month_sin'] = np.sin(2 * np.pi * train_df['month'] / 12)
train_df['month_cos'] = np.cos(2 * np.pi * train_df['month'] / 12)
train_df['day_sin'] = np.sin(2 * np.pi * train_df['day'] / 31)
train_df['day_cos'] = np.cos(2 * np.pi * train_df['day'] / 31)

# Thêm các đặc trưng sin và cos cho test_df
test_df["day"] = test_df["date"].dt.day
test_df["month"] = test_df["date"].dt.month
test_df["year"] = test_df["date"].dt.year
test_df['year_sin'] = np.sin(2 * np.pi * test_df['year'] / train_df['year'].max())  # Sử dụng max từ train_df
test_df['year_cos'] = np.cos(2 * np.pi * test_df['year'] / train_df['year'].max())  # Sử dụng max từ train_df
test_df['month_sin'] = np.sin(2 * np.pi * test_df['month'] / 12)
test_df['month_cos'] = np.cos(2 * np.pi * test_df['month'] / 12)
test_df['day_sin'] = np.sin(2 * np.pi * test_df['day'] / 31)
test_df['day_cos'] = np.cos(2 * np.pi * test_df['day'] / 31)

# Chuẩn hóa các đặc trưng sin và cos
cols_to_scale = ['year_sin', 'year_cos', 'month_sin', 'month_cos', 'day_sin', 'day_cos']
scaler = MinMaxScaler()
train_df[cols_to_scale] = scaler.fit_transform(train_df[cols_to_scale])
test_df[cols_to_scale] = scaler.transform(test_df[cols_to_scale])






#Tuong tac giua warehouse và holiday_name
train_df['warehouse_holiday'] = (train_df['warehouse'].astype(str) + '_' + train_df['holiday_name'].astype(str)).astype('category')
test_df['warehouse_holiday'] = (test_df['warehouse'].astype(str) + '_' + test_df['holiday_name'].astype(str)).astype('category')
# Mã hóa lại
le = LabelEncoder()
combined = np.concatenate([train_df['warehouse_holiday'].astype(str), test_df['warehouse_holiday'].astype(str)])
le.fit(combined)
train_df['warehouse_holiday'] = le.transform(train_df['warehouse_holiday'].astype(str)).astype(np.int32)
test_df['warehouse_holiday'] = le.transform(test_df['warehouse_holiday'].astype(str)).astype(np.int32)







#Lag của total_orders (orders_lag):
for lag in [7, 14, 28]:
    train_df[f'orders_lag_{lag}'] = train_df.groupby(['unique_id', 'warehouse'])['total_orders'].shift(lag).astype(np.float32)
    test_df[f'orders_lag_{lag}'] = test_df.groupby(['unique_id', 'warehouse'])['total_orders'].shift(lag).astype(np.float32)




# Kiểm tra kết quả
print("Cột trong train_df sau khi thêm đặc trưng:", train_df.columns.tolist())
print("Cột trong test_df sau khi thêm đặc trưng:", test_df.columns.tolist())

In [ ]:
# keep_columns.append('category_sales_sum')
# keep_columns.append('category_sales_avg')
# keep_columns.append('category_orders_sum')
# keep_columns.append('category_orders_avg')
# keep_columns.append('total_discount')
# keep_columns.append('L1_category_name_en')
# train_df = train_df[keep_columns]
train_df.columns

In [ ]:
cols_to_scale = ['category_sales_sum', 'category_sales_avg','category_orders_sum', 'category_orders_avg']

scaler = MinMaxScaler()
train_df[cols_to_scale] = scaler.fit_transform(train_df[cols_to_scale])

In [ ]:


# Định nghĩa lag_features (có thể điều chỉnh)
lag_features = [1, 7, 14, 28]  # Lag 1, 2, 3, 7 ngày



# Đảm bảo cột date là datetime
train_df['date'] = pd.to_datetime(train_df['date'])
test_df['date'] = pd.to_datetime(test_df['date'])

# --- Xử lý train_df ---

# Nhóm dữ liệu theo unique_id và warehouse
df_grouped = train_df.groupby(["unique_id", "warehouse"])

# Tính lag features cho train_df
for i in lag_features:
    # Tạo cột lag
    train_df[f"sales_item_warehouse_lag_{i}"] = df_grouped["sales"].shift(i)
    # Điền NaN bằng giá trị sales cuối cùng của nhóm, nếu không có thì điền 0
    train_df[f"sales_item_warehouse_lag_{i}"] = train_df[f"sales_item_warehouse_lag_{i}"].fillna(
        df_grouped["sales"].transform("last")
    ).fillna(0)

# Kiểm tra NaN trong các cột lag của train_df
print("\nKiểm tra NaN trong các cột lag của train_df:")
print(train_df[[f"sales_item_warehouse_lag_{i}" for i in lag_features]].isna().sum())

# Kiểm tra phân bố của các cột lag
print("\nThống kê mô tả cho các cột lag trong train_df:")
print(train_df[[f"sales_item_warehouse_lag_{i}" for i in lag_features]].describe())

# --- Xử lý test_df ---

# Tạo DataFrame kết hợp train_df và test_df để tính lag liên tục
# Chỉ lấy các cột cần thiết từ train_df
train_subset = train_df[["date", "unique_id", "warehouse", "sales"]].copy()
# Tạo cột sales giả cho test_df (đặt là NaN vì test_df không có sales)
test_subset = test_df[["date", "unique_id", "warehouse"]].copy()
test_subset["sales"] = np.nan

# Kết hợp train_df và test_df
combined_df = pd.concat([train_subset, test_subset], ignore_index=True)
combined_df = combined_df.sort_values(["unique_id", "warehouse", "date"])

# Nhóm dữ liệu kết hợp
combined_grouped = combined_df.groupby(["unique_id", "warehouse"])

# Tính lag features cho combined_df
for i in lag_features:
    combined_df[f"sales_item_warehouse_lag_{i}"] = combined_grouped["sales"].shift(i)
    # Điền NaN bằng giá trị sales cuối cùng của nhóm, nếu không có thì điền 0
    combined_df[f"sales_item_warehouse_lag_{i}"] = combined_df[f"sales_item_warehouse_lag_{i}"].fillna(
        combined_grouped["sales"].transform("last")
    ).fillna(0)

# Lấy các cột lag cho test_df
test_lag_features = combined_df[combined_df["date"].isin(test_df["date"])][
    ["date", "unique_id", "warehouse"] + [f"sales_item_warehouse_lag_{i}" for i in lag_features]
]

# Gộp lag features vào test_df
test_df = test_df.merge(
    test_lag_features[["date", "unique_id"] + [f"sales_item_warehouse_lag_{i}" for i in lag_features]],
    on=["date", "unique_id"],
    how="left"
)

# Kiểm tra NaN trong các cột lag của test_df
print("\nKiểm tra NaN trong các cột lag của test_df:")
print(test_df[[f"sales_item_warehouse_lag_{i}" for i in lag_features]].isna().sum())

# Kiểm tra phân bố của các cột lag trong test_df
print("\nThống kê mô tả cho các cột lag trong test_df:")
print(test_df[[f"sales_item_warehouse_lag_{i}" for i in lag_features]].describe())

# Lưu dữ liệu
train_df.to_csv('train_df_with_lags.csv', index=False)
test_df.to_csv('test_df_with_lags.csv', index=False)

In [ ]:
train_df.info()

In [ ]:
plt.figure(figsize=(16, 12))
corr_matrix = train_df.corr(numeric_only=True).abs()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.show()

In [ ]:

# Phân tích tương quan với sales
corr_with_sales = corr_matrix['sales'].abs().sort_values(ascending=False)
print("Tương quan tuyệt đối với sales:")
print(corr_with_sales)

# Vẽ biểu đồ phân phối tương quan
plt.figure(figsize=(10, 6))
plt.hist(corr_with_sales.drop('sales'), bins=20)
plt.xlabel('Tương quan tuyệt đối với sales')
plt.ylabel('Số lượng đặc trưng')
plt.title('Phân phối tương quan với sales')
plt.show()



In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoders = {}
for feature in ["warehouse", "holiday_name"]:
    le = LabelEncoder()
    df[feature] = le.fit_transform(df[feature])
    label_encoders[feature] = le

In [ ]:
    # Loại bỏ cột weight và availability
    columns_to_drop = ['weight', 'availability']
    train_df = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns], errors='ignore')

In [ ]:
"""
Note 12:
    Not ideal for all solutions, but didn't explore this in depth yet
"""
train_df["sales"] = train_df["sales"].fillna(0)
train_df["warehouse_demand"] = train_df["warehouse_demand"].fillna(0)
train_df = train_df.fillna(0)

In [ ]:
!pip install catboost

In [ ]:
# Giả sử train_df, test_df, weights_df đã được chuẩn bị
# Tối ưu hóa kiểu dữ liệu (như trong pipeline)
for col in train_df.select_dtypes('float64'):
    train_df[col] = train_df[col].astype('float32')
for col in train_df.select_dtypes('int64'):
    train_df[col] = train_df[col].astype('int32')
for col in train_df.select_dtypes('bool'):
    train_df[col] = train_df[col].astype('uint8')

for col in test_df.select_dtypes('float64'):
    test_df[col] = test_df[col].astype('float32')
for col in test_df.select_dtypes('int64'):
    test_df[col] = test_df[col].astype('int32')
for col in test_df.select_dtypes('bool'):
    test_df[col] = test_df[col].astype('uint8')
# Tiền xử lý các cột object
categorical_cols = ['warehouse', 'holiday_name', 'name', 'L1_category_name_en', 
                    'L2_category_name_en', 'L3_category_name_en', 'L4_category_name_en']
# Áp dụng Label Encoding, giảm bản sao
label_encoders = {}
for col in categorical_cols:
    if col in train_df.columns and col in test_df.columns:
        le = LabelEncoder()
        # Xử lý từng cột để giảm RAM
        train_vals = train_df[col].astype(str).values
        test_vals = test_df[col].astype(str).values
        combined = np.concatenate([train_vals, test_vals])
        le.fit(combined)
        train_df[col] = le.transform(train_vals)
        test_df[col] = le.transform(test_vals)
        label_encoders[col] = le
        del combined  # Xóa biến tạm

# # Hàm inverse_norm
# def inverse_norm(df, period, values):
#     sales_min = train_df["sales"].min()
#     sales_max = train_df["sales"].max()
#     return values * (sales_max - sales_min) + sales_min


# Định nghĩa features dựa trên cả train_df và test_df
# Loại bỏ cột datetime64 và chỉ giữ cột số
train_features = [c for c in train_df.columns if c not in ["unique_id", "date", "sales", "availability"] 
                 and train_df[c].dtype in [np.float32, np.int32, np.uint8, np.int64]]
test_features = [c for c in test_df.columns if c not in ["unique_id", "date", "sales", "availability"] 
                 and test_df[c].dtype in [np.float32, np.int32, np.uint8, np.int64]]
# Lấy tập hợp chung của features
features = list(set(train_features) & set(test_features))

# Kiểm tra kiểu dữ liệu của features
print("Feature dtypes in train_df:")
for col in features:
    print(f"{col}: {train_df[col].dtype}")
print("\nFeature dtypes in test_df:")
for col in features:
    print(f"{col}: {test_df[col].dtype}")


target = "sales"
training_dates = (train_df["date"].min(), train_df["date"].max() - pd.Timedelta(days=28))
validation_dates = (training_dates[1] + pd.Timedelta(days=1), training_dates[1] + pd.Timedelta(days=14))
test_dates = (test_df["date"].min(), test_df["date"].max())
weight_map = weights_df.set_index('unique_id')['weight'].to_dict()

# Chuẩn bị dữ liệu
X_train = train_df[train_df["date"].between(*training_dates)][features]
y_train = train_df[train_df["date"].between(*training_dates)][target]
X_val = train_df[train_df["date"].between(*validation_dates)][features]
y_val = train_df[train_df["date"].between(*validation_dates)][target]
unique_id_val = train_df[train_df["date"].between(*validation_dates)]["unique_id"]
# Kiểm tra NaN
if X_train.isna().any().any() or X_val.isna().any().any():
    print("NaN detected in input data. Filling remaining NaNs with mean.")
    X_train = X_train.fillna(X_train.mean())
    X_val = X_val.fillna(X_val.mean())# Kiểm tra NaN
if X_train.isna().any().any() or X_val.isna().any().any():
    print("NaN detected in input data. Filling remaining NaNs with mean.")
    X_train = X_train.fillna(X_train.mean())
    X_val = X_val.fillna(X_val.mean())
# Kiểm tra weight_map
missing_ids = set(unique_id_val) - set(weight_map.keys())
if missing_ids:
    print(f"Missing unique_id in weight_map: {missing_ids}")
    # Gán trọng số mặc định = 1 cho các unique_id thiếu
    for missing_id in missing_ids:
        weight_map[missing_id] = 1.0

In [ ]:
# Chuyển dữ liệu sang GPU
X_train_gpu = cp.array(X_train.values, dtype=cp.float32)
y_train_gpu = cp.array(y_train.values, dtype=cp.float32)
X_val_gpu = cp.array(X_val.values, dtype=cp.float32)

# Huấn luyện mô hình XGBoost với GridSearchCV và GPU
xgb = XGBRegressor(
    objective='reg:absoluteerror',
    random_state=2025,
    use_label_encoder=False,
    tree_method='hist',
    device='cuda'  # Chạy trên GPU
)
xgb_param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.05, 0.1]
}
xgb_gs = GridSearchCV(
    xgb,
    xgb_param_grid,
    cv=3,
    scoring='neg_mean_absolute_error',
    n_jobs=1,  # Tắt song song để giảm RAM
    verbose=1
)

# Huấn luyện và kiểm tra lỗi
try:
    xgb_gs.fit(X_train_gpu.get(), y_train_gpu.get())  # Chuyển về numpy cho GridSearchCV
    print("GridSearchCV completed successfully.")
except Exception as e:
    print(f"Error during GridSearchCV fit: {e}")
    raise



y_pred_xgb = cp.asnumpy(xgb_gs.predict(X_val_gpu)) 

# Đánh giá mô hình
wmae = mean_absolute_error(y_val, y_pred_xgb, 
                          sample_weight=unique_id_val.map(weight_map).values)

# In kết quả
print(f"Best Parameters: {xgb_gs.best_params_}")
print(f"Validation WMAE: {wmae}")
print(f"Test Predictions: {y_pred_xgb}")

In [ ]:
# === 5. Ridge Regression ===
ridge_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('ridge', Ridge())
])
ridge_param_grid = {
    'ridge__alpha': [0.1, 1, 10, 100],
    'ridge__solver': ['auto', 'lsqr'],
    'ridge__fit_intercept': [True]
}
ridge_gs = GridSearchCV(
    ridge_pipe,
    ridge_param_grid,
    cv=5,
    n_jobs=1,
    scoring='neg_mean_absolute_error', 
    verbose=1
)
ridge_gs.fit(X_train, y_train)
# Dự đoán trên tập xác thực và kiểm tra
y_pred_ridge = ridge_gs.predict(X_val)

# Đánh giá mô hình
wmae = mean_absolute_error(y_val, y_pred_ridge, 
                          sample_weight=unique_id_val.map(weight_map).values)

# In kết quả
print(f"Best Parameters: {ridge_gs.best_params_}")
print(f"Validation WMAE: {wmae}")
print(f"Test Predictions: {y_pred_ridge}")

In [ ]:
# Chuyển dữ liệu sang GPU
X_train_np = X_train.values.astype(np.float32)
y_train_np = y_train.values.astype(np.float32)
X_val_np = X_val.values.astype(np.float32)



# === 6. LightGBM (GridSearch) ===
lgb = LGBMRegressor(objective= 'regression', random_state=43, force_row_wise=True, verbose=-1, device='gpu')
lgb_param_dist = {
    'n_estimators': [1000],
    'max_depth': [10],
    'learning_rate': [0.054],
    'num_leaves': [273],
    'min_child_samples': [40],
    'colsample_bytree' : [0.852], 
    'colsample_bynode' : [0.72], 
    'min_data_in_leaf' : [6],
    'reg_alpha': [0.056],
    'num_boost_round': [11000],
    'reg_lambda': [0.4]
}
lgb_gs = GridSearchCV(
    lgb, lgb_param_dist,
    cv=5,
    scoring='neg_mean_absolute_error',
    n_jobs=1, verbose=1)
# Huấn luyện
try:
    lgb_gs.fit(X_train_np, y_train_np)  # Chuyển về numpy cho RandomizedSearchCV
    print("GridSearchCV completed successfully.")
except Exception as e:
    print(f"Error during RandomizedSearchCV fit: {e}")
    raise


y_pred_lgb = cp.asnumpy(lgb_gs.predict(X_val_np)) 
# Đánh giá mô hình
wmae = mean_absolute_error(y_val, y_pred_lgb, 
                          sample_weight=unique_id_val.map(weight_map).values)

# In kết quả
print(f"Best Parameters: {lgb_gs.best_params_}")
print(f"Validation WMAE: {wmae}")
print(f"Test Predictions: {y_pred_lgb}")

In [ ]:
# === 7. ElasticNet ===
en_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('enet', ElasticNet(random_state=43))
])
en_param_grid = {
    'enet__alpha': [0.01, 0.1, 1],
    'enet__l1_ratio': [0.1, 0.5, 0.9],
    'enet__fit_intercept': [True, False],
    'enet__max_iter': [1000]
}
en_gs = GridSearchCV(
    en_pipe,
    en_param_grid,
    cv=3,
    scoring='neg_mean_absolute_error',
    n_jobs=1,
    verbose=1
)
en_gs.fit(X_train, y_train)
y_pred_en = en_gs.predict(X_val)
# Đánh giá mô hình
wmae = mean_absolute_error(y_val, y_pred_en, 
                          sample_weight=unique_id_val.map(weight_map).values)

# In kết quả
print(f"Best Parameters: {en_gs.best_params_}")
print(f"Validation WMAE: {wmae}")
print(f"Test Predictions: {y_pred_en}")


In [ ]:
#8. catboost
# Huấn luyện mô hình CatBoost với GridSearchCV và GPU
X_train_np = X_train.values.astype(np.float32)
y_train_np = y_train.values.astype(np.float32)
X_val_np = X_val.values.astype(np.float32)


# Huấn luyện mô hình CatBoost với GridSearchCV và GPU
cat = CatBoostRegressor(random_seed=43, verbose=0, task_type='GPU', grow_policy='Lossguide')
cat_param_grid = {
    'iterations': [500, 1000],
    'depth': [8, 10],
    'bagging_temperature': [0.5],
    'learning_rate': [0.05, 0.1],
    'max_leaves': [128],
    'l2_leaf_reg': [2],
    'min_data_in_leaf': [24]
}
cat_gs = GridSearchCV(
    cat,
    cat_param_grid,
    cv=5,
    scoring='neg_mean_absolute_error',
    n_jobs=1,  
    verbose=1
)


try:
    cat_gs.fit(X_train_np, y_train_np) 
    print("GridSearchCV completed successfully.")
except Exception as e:
    print(f"Error during RandomizedSearchCV fit: {e}")
    raise

y_pred_cat = cat_gs.predict(X_val_np)
# Đánh giá mô hình
wmae = mean_absolute_error(y_val, y_pred_cat, 
                          sample_weight=unique_id_val.map(weight_map).values)

# In kết quả
print(f"Best Parameters: {cat_gs.best_params_}")
print(f"Validation WMAE: {wmae}")
print(f"Test Predictions: {y_pred_cat}")

In [ ]:
keep_columns_1=train_df.columns
keep_columns_1

In [ ]:
train_df.info()

In [ ]:
test_df.info()

In [ ]:
# 1. Identify missing columns
missing_cols = list(set(train_df.columns) - set(test_df.columns))
missing_cols

In [ ]:
keep_columns=train_df.columns
keep_columns

In [ ]:
# # 2. Mã hóa và chuyển đổi date
# le = LabelEncoder()
# # test_df['L1_category_name_en'] = le.fit_transform(test_df['L1_category_name_en'])
# # test_df['holiday_name'] = le.fit_transform(test_df['holiday_name'])  # Apply Label Encoding
# test_df['name'] = le.fit_transform(test_df['name'])
# test_df['L2_category_name_en'] = le.fit_transform(test_df['L2_category_name_en'].astype(str))
# test_df['L3_category_name_en'] = le.fit_transform(test_df['L3_category_name_en'].astype(str))
# test_df['L4_category_name_en'] = le.fit_transform(test_df['L4_category_name_en'].astype(str))


# # test_df['date'] = pd.to_datetime(test_df['date'])
# # test_df['date'] = (test_df['date'] - test_df['date'].min()).dt.days
# for col in test_df.select_dtypes('float64'): test_df[col] = test_df[col].astype('float32')
# for col in test_df.select_dtypes('int64'):   test_df[col] = test_df[col].astype('int32')
# for col in test_df.select_dtypes('bool'):    test_df[col] = test_df[col].astype('uint8')

In [ ]:
# 2. Add missing columns to test_df with default values (e.g., 0)
for col in missing_cols:
    test_df[col] = 0  # Or use np.nan or another suitable default

# 3. Select the desired columns
test_df = test_df[keep_columns]
test_df.info()

In [ ]:
# Dự đoán trên test_df
try:
    X_test_new = test_df[test_df["date"].between(*test_dates)][features]  # Lọc theo test_dates
    X_test_new_np = X_test_new.values.astype(np.float32)
    y_pred_test = lgb_gs.predict(X_test_new_np)
    # Nếu cần inverse_norm, bỏ comment dòng dưới
    # y_pred_test = inverse_norm(test_df, y_pred_test)
    test_df.loc[test_df["date"].between(*test_dates), 'sales'] = y_pred_test
except Exception as e:
    print(f"Error during test prediction: {e}")
    raise

In [ ]:
solution = pd.DataFrame({
    'id': df5['id'],       # id từ df5
    'sales_hat': test_df['sales']  # sales từ test_df
})

# Lưu kết quả vào file CSV
solution.to_csv('submission.csv', index=False)


In [ ]:
print(solution)